# Using Bonito Model to Generate Q&A Pairs

Instruction tuning is a method to increase the instruction following capabilities and general zero-shot performance of LLMs. Using synthetic data in this task is widely used (Self-Instruct, OpenOrca, etc.). Nayak et al. {cite}`nayak_learning_2024` introduced [Bonito](https://huggingface.co/BatsResearch/bonito-v1) model for conditional task generation which enables converting plain (unannotated) text to instruction-tuning datasets. We will use this model to develop a "critical infrastructure assessor LLM" using the MOSIP documentation and Okta's recent threat intelligence report as unannotated datasets.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
from pathlib import Path
cache_path = Path('/bask/projects/v/vjgo8416-ai-fairness/.cache/huggingface')

In [4]:
tokenizer = AutoTokenizer.from_pretrained("BatsResearch/bonito-v1", cache_dir=cache_path)
model = AutoModelForCausalLM.from_pretrained("BatsResearch/bonito-v1", cache_dir=cache_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/bask/homes/f/fscd8270/.local/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
from datasets import Dataset,load_from_disk
mosipds = load_from_disk("datasets/mosip_dataset.hf")

In [6]:
#input_text = mosipds[1]["input"]
#input_text = "Summarise the following text: " + input_text 

In [7]:
#input_ids = tokenizer(input_text, return_tensors="pt")
#outputs = model.generate(**input_ids, max_new_tokens=30)
#print(tokenizer.decode(outputs[0]))

In [8]:
SHORTFORM_TO_FULL_TASK_TYPES = {
    "exqa": "extractive question answering",
    "mcqa": "multiple-choice question answering",
    "qg": "question generation",
    "qa": "question answering without choices",
    "ynqa": "yes-no question answering",
    "coref": "coreference resolution",
    "paraphrase": "paraphrase generation",
    "paraphrase_id": "paraphrase identification",
    "sent_comp": "sentence completion",
    "sentiment": "sentiment",
    "summarization": "summarization",
    "text_gen": "text generation",
    "topic_class": "topic classification",
    "wsd": "word sense disambiguation",
    "te": "textual entailment",
    "nli": "natural language inference",
}

In [9]:
def prepare_bonito_input(context_dataset: Dataset, task_type: str, context_col: str) -> Dataset:
        """
        Prepares the input for the Bonito model.

        This method takes a context dataset, a task type, and a context
        column name, and prepares the dataset for the Bonito model.
        If the task type is not recognized, it raises a ValueError.

        Args:
            context_dataset (Dataset): The dataset that provides the
                context for the task.
            task_type (str): The type of the task. This can be a
                short form or a full form. If the task type is not
                recognized, a ValueError is raised.
            context_col (str): The name of the column in the dataset
                that provides the context for the task.
            **kwargs: Additional keyword arguments.

        Returns:
            Dataset: The prepared dataset for the Bonito model.
        """
        # get the task type name
        if task_type in SHORTFORM_TO_FULL_TASK_TYPES.values():
            full_task_type = task_type
        elif task_type in SHORTFORM_TO_FULL_TASK_TYPES:
            full_task_type = SHORTFORM_TO_FULL_TASK_TYPES[task_type]
        else:
            raise ValueError(f"Task type {task_type} not recognized")

        def process(example):
            input_text = "<|tasktype|>\n" + full_task_type.strip()
            input_text += (
                "\n<|context|>\n" + example[context_col].strip() + "\n<|task|>\n"
            )
            return {
                "input": input_text,
            }

        return context_dataset.map(
            process,
            remove_columns=context_dataset.column_names,
            num_proc=1,
        )

In [34]:
def postprocess_dataset(synthetic_dataset: Dataset, context_col: str) -> Dataset:
        """
        Post-processes the synthetic dataset.

        This method takes a synthetic dataset and a context column
        name, and post-processes the dataset. It filters out
        examples where the prediction does not contain exactly two
        parts separated by "<|pipe|>", and then maps each example to a
        new format where the context is inserted into the first part of
        the prediction and the second part of the prediction is used as
        the output.

        Args:
            synthetic_dataset (Dataset): The synthetic dataset to be
                post-processed.
            context_col (str): The name of the column in the dataset
                that provides the context for the tasks.
            **kwargs: Additional keyword arguments.

        Returns:
            Dataset: The post-processed synthetic dataset.
        """
        synthetic_dataset = synthetic_dataset.filter(
            lambda example: len(example["prediction"].split("<|pipe|>")) == 2
        )

        def process(example):
            pair = example["prediction"].split("<|pipe|>")
            context = example[context_col].strip()
            return {
                "input": pair[0].strip().replace("{{context}}", context),
                "output": pair[1].strip(),
            }

        synthetic_dataset = synthetic_dataset.map(
            process,
            remove_columns=synthetic_dataset.column_names,
            num_proc=1,
        )

        return synthetic_dataset

In [16]:
def convert_to_dataset(text):
    dataset = Dataset.from_list([{"input": text}])
    return dataset

In [17]:
# If you would like to test the model with only one simple text, you can uncomment the following code:

# unannotated_paragraph = """1. “Confidential Information”, whenever used in this Agreement, shall mean any data, document, specification and other information or material, that is delivered or disclosed by UNHCR to the Recipient in any form whatsoever, whether orally, visually in writing or otherwise (including computerized form), and that, at the time of disclosure to the Recipient, is designated as confidential."""

# processed_dataset = prepare_bonito_input(
#    context_dataset=convert_to_dataset(unannotated_paragraph),
#    context_col="input",
#    task_type="nli"
# )

In [23]:
processed_dataset = prepare_bonito_input(
    context_dataset=mosipds,
    context_col="input",
    task_type="nli"
)

In [24]:
processed_dataset[0]

{'input': '<|tasktype|>\nnatural language inference\n<|context|>\nOverview - Multiple language support : \n* Registration Client is featured to allow an operator to choose the operation language. Option to select their preferred language is provided on the login screen.\n* Data collection during registration client supports more than one language at a time.\n* Before starting any registration process, the operator can choose the languages amongst the configured ones.\n \n  \nTo know more about setting up the reference registration client, refer to [Registration Client Installation Guide](https://docs.mosip.io/1.2.0/modules/registration-client/registration-client-installation-guide).\n\nTo know more about the features present in the Registration Client, refer to [Registration Client User Guide](https://docs.mosip.io/1.2.0/modules/registration-client/registration-client-user-guide).\n<|task|>\n'}

In [30]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# collect multiple generations into one dataset object
examples = []

for i, example in enumerate(mosipds.to_list()):
    input_ids = tokenizer(processed_dataset["input"][i], return_tensors="pt", padding=True)
    outputs = model.generate(**input_ids, max_new_tokens=256, top_p=0.95, temperature=0.7)
    examples.append(
        {"context": "input", "prediction": tokenizer.decode(outputs[0])}
    )
    print(examples[-1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': 'input', 'prediction': '<s> <|tasktype|>\nnatural language inference\n<|context|>\nOverview - Multiple language support : \n* Registration Client is featured to allow an operator to choose the operation language. Option to select their preferred language is provided on the login screen.\n* Data collection during registration client supports more than one language at a time.\n* Before starting any registration process, the operator can choose the languages amongst the configured ones.\n \n  \nTo know more about setting up the reference registration client, refer to [Registration Client Installation Guide](https://docs.mosip.io/1.2.0/modules/registration-client/registration-client-installation-guide).\n\nTo know more about the features present in the Registration Client, refer to [Registration Client User Guide](https://docs.mosip.io/1.2.0/modules/registration-client/registration-client-user-guide).\n<|task|>\n {{context}} Based on the previous passage, is it true that "The R

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': 'input', 'prediction': '<s> <|tasktype|>\nnatural language inference\n<|context|>\nOverview - Who operates the Registration Client? : \nThe Registration Client can be operated by an operator who can be either a **Supervisor** or an **Officer**. They can login to the client application and perform various activities. The Supervisor and the Officer can perform tasks like Onboarding, Synchronize Data, Upgrade software, Export packet, Upload packets, View Re-registration packets, Correction process, Exception authentication, etc. In addition to this, the Supervisor has exclusive authority to Approve/reject registrations.\n\nTo know more about the onboarding process of an operator, refer to [Operator onboarding](operator-onboarding.md).\n<|task|>\n {{context}} Based on the previous passage, is it true that "The Registration Client can be operated by an operator who can be either a **Supervisor** or an **Officer**. They can login to the client application and perform various acti

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

{'context': 'input', 'prediction': '<s> <|tasktype|>\nnatural language inference\n<|context|>\n🌍 Country Implementation - Overview : \nThis is a guide to implement MOSIP for a country. It is advised that Government and System Integrators (SI) study the recommended steps to work out an appropriate implementation strategy. The items are in "near-chronological order" and may differ for an implementation.\n<|task|>\n {{context}} Based on the previous passage, is it true that "The items are in "near-chronological order" and may differ for an implementation. The items are in "near-chronological order" and may differ for an implementation. The items are in "near-chronological order" and may differ for an implementation. "? Yes, no, or maybe?\n<|pipe|>\nYes</s>'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': 'input', 'prediction': '<s> <|tasktype|>\nnatural language inference\n<|context|>\n🌍 Country Implementation - Key decisions : \n1. Choice of deployment of [Pre-registration](id-lifecycle-management.md#pre-registration).\n2. Rate of enrolment desired.\n3. Rate of authentication expected.\n4. [Languages](module-configuration.md#languages).\n5. Customisation and procurement of components as given [here](reference-implementations.md).\n6. [ID schema](id-schema.md) (as prescribed by the country\'s regulatory authority).\n7. Hardware requirements estimate.\n   * [Server side](https://github.com/mosip/documentation/tree/develop/docs/\\_files)\n   * [Devices](\\_files/mosip-devices-calculator.xlsx)\n8. [Credential choices](id-repository.md#credential-types).\n9. ID Card print design.\n10. MOSIP versions.\n11. MOSIP support (scope).\n12. Disaster recovery strategy.\n13. Phased approach for rollout.\n<|task|>\n {{context}} Based on the previous passage, is it true that "The country\'

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': 'input', 'prediction': '<s> <|tasktype|>\nnatural language inference\n<|context|>\n🌍 Country Implementation - Procurement : \n1. Engagement with an SI - terms and conditions.\n2. Procurement of biometric and other external components.\n3. HSM\n4. [Postgres](https://docs.mosip.io/1.2.0/modules/persistence/postgres-db))\n5. [Object store](https://docs.mosip.io/1.2.0/modules/persistence/object-store)\n6. Compute hardware\n<|task|>\n {{context}} Based on the previous passage, is it true that "The terms and conditions are not very long."? Yes, no, or maybe?\n<|pipe|>\nMaybe</s>'}


In [ ]:
synthetic_dataset = Dataset.from_list(examples)

# filter out the examples that cannot be parsed
synthetic_dataset = postprocess_dataset(
    synthetic_dataset, context_col="context"
)

synthetic_dataset.save_to_disk("datasets/mosip_bonito.hf")